In [35]:
import nltk
nltk.download('punkt')
nltk.download('brown')
from nltk import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import brown
from nltk.corpus import indian
nltk.download('indian')
import collections

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package indian to /root/nltk_data...
[nltk_data]   Package indian is already up-to-date!


In [36]:
possible_tags = list()
all_words = list()
observations = list()
tag_dict = dict()
starter_dict =  dict()
tx_matrix = list()
emission_matrix = dict()
tag_words = dict()
sentences = [[('the','Det'),('students','N'),('pass','V'),('the','Det'),('test','N'),('.','PUNCT')],[('the','Det'),('students','N'),('wait','V'),('for','P'),('the','Det'),('pass','N'),('.','PUNCT')],[('teachers','N'),('test','V'),('students','N'),('.','PUNCT')]]
possible_tags = ['Det','N','V','PUNCT','P']
for tag in possible_tags:
  tag_dict[tag] = 0
  tag_words[tag] = list()
  starter_dict[tag] = 0
for sentence in sentences:
  for word in sentence:
    tag_dict[word[1]] += 1
    tag_words[word[1]].append(word[0])
for tag,lists in tag_words.items():
  tag_words[tag] = dict(collections.Counter(lists))
#emission probability - only include non zero values
for tag,words in tag_words.items():
  for word,count in words.items():
    new_count = count/tag_dict[tag]
    words[word] = new_count
print(tag_dict)
print(tag_words)

#starting probability - 
denominator = len(sentences)
for sentence in sentences:
  starter_dict[sentence[0][1]]+=1
for tag,counts in starter_dict.items():
  starter_dict[tag] = counts/denominator
print(starter_dict)

#transition probability - 
pair_counts = dict()
for tag in possible_tags:
  pair_counts[tag] = dict()
  for other_tag in possible_tags:
    pair_counts[tag][other_tag] = 0
#print(pair_counts)
for sentence in sentences:
  for i in range(1,len(sentence)):
    pair_counts[sentence[i-1][1]][sentence[i][1]] += 1
print(pair_counts)
for current_tag,next_tag_dict in pair_counts.items():
  #print(type(next_tag_dict),next_tag_dict)
  for tag,count in next_tag_dict.items():
    new_count = count/tag_dict[current_tag]
    next_tag_dict[tag] = new_count
print(pair_counts)


{'Det': 4, 'N': 6, 'V': 3, 'PUNCT': 3, 'P': 1}
{'Det': {'the': 1.0}, 'N': {'students': 0.5, 'test': 0.16666666666666666, 'pass': 0.16666666666666666, 'teachers': 0.16666666666666666}, 'V': {'pass': 0.3333333333333333, 'wait': 0.3333333333333333, 'test': 0.3333333333333333}, 'PUNCT': {'.': 1.0}, 'P': {'for': 1.0}}
{'Det': 0.6666666666666666, 'N': 0.3333333333333333, 'V': 0.0, 'PUNCT': 0.0, 'P': 0.0}
{'Det': {'Det': 0, 'N': 4, 'V': 0, 'PUNCT': 0, 'P': 0}, 'N': {'Det': 0, 'N': 0, 'V': 3, 'PUNCT': 3, 'P': 0}, 'V': {'Det': 1, 'N': 1, 'V': 0, 'PUNCT': 0, 'P': 1}, 'PUNCT': {'Det': 0, 'N': 0, 'V': 0, 'PUNCT': 0, 'P': 0}, 'P': {'Det': 1, 'N': 0, 'V': 0, 'PUNCT': 0, 'P': 0}}
{'Det': {'Det': 0.0, 'N': 1.0, 'V': 0.0, 'PUNCT': 0.0, 'P': 0.0}, 'N': {'Det': 0.0, 'N': 0.0, 'V': 0.5, 'PUNCT': 0.5, 'P': 0.0}, 'V': {'Det': 0.3333333333333333, 'N': 0.3333333333333333, 'V': 0.0, 'PUNCT': 0.0, 'P': 0.3333333333333333}, 'PUNCT': {'Det': 0.0, 'N': 0.0, 'V': 0.0, 'PUNCT': 0.0, 'P': 0.0}, 'P': {'Det': 1.0, 'N':

In [47]:
test = 'the teachers pass the students.'
word_list = word_tokenize(test)
print(word_list)

['the', 'teachers', 'pass', 'the', 'students', '.']


In [63]:
#viterbi matrix
word_values_dict = dict()
for i in range(len(word_list)):
  word_values_dict[i] = dict()
  for tag in possible_tags:
    word_values_dict[i][tag] = (0,'.')

for i in range(len(word_list)):
  current_word = word_list[i]
  #print(current_word)
  for tag in possible_tags:
    if current_word not in tag_words[tag].keys():
      #word_values_dict[i][tag] = (0,'q0')
      continue
    emission_prob = tag_words[tag][current_word]
    transition_prob = starter_dict[tag]
    if i != 0:
      max_val = 0
      max_state = '.'
      for prev_tag in possible_tags:
        curr_val = word_values_dict[i-1][prev_tag][0]*pair_counts[prev_tag][tag] #worried
        if curr_val>max_val:
          max_val = curr_val
          max_state = prev_tag
      word_values_dict[i][tag] = (emission_prob*max_val,max_state)
      
    else:
      word_values_dict[i][tag] = (emission_prob*transition_prob,'q0')

answer = list()
last_dist = word_values_dict[len(word_list)-1]
#print(last_dist)
max_tag = '.'
max_val = 0
for tag in possible_tags:
  if last_dist[tag][0]>max_val:
    max_val = last_dist[tag][0]
    max_tag = tag
answer.append(max_tag)
prev_tag = last_dist[max_tag][1]
answer.append(prev_tag)
for i in range(len(word_list)-2,-1,-1):
  next_prev = word_values_dict[i][prev_tag][1]
  answer.append(next_prev)
  prev_tag = next_prev
answer.reverse()
print(word_list)
print(answer[1:])

['the', 'teachers', 'pass', 'the', 'students', '.']
['Det', 'N', 'V', 'Det', 'N', 'PUNCT']
